In [1]:
#!pip install 'monai[all]'

In [2]:
#Thank god-> Starting from Python 3.7, dictionaries maintain the insertion order.

import importlib
import subprocess

def check_and_install_packages(package_list):
    missing_packages = []

    for package in package_list:
        try:
            __import__(package)
        except ImportError:
            missing_packages.append(package)

    if missing_packages:
        print("Installing missing packages:")
        for package in missing_packages:
            subprocess.run(["pip", "install", package])
            print(f"Installed {package}")
    else:
        print("All required packages are already installed.")

# List of packages to check and install if necessary
required_packages = [
    "numpy",
    "matplotlib",
    "pandas",
    #"cv2",
    "imageio",
    "PIL",
    "torch",
    "monai",
    "sys"
]

check_and_install_packages(required_packages)

In [3]:
import sys
print(sys.version)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import pandas as pd
import os
import cv2 as cv
import imageio
from PIL import Image
import imageio.v3 as iio

In [6]:
#updated_raw_dict, new_analyzed_dict
import PIL

In [7]:
def convert_to_numpy(data):
    if isinstance(data, np.ndarray):
        return data.astype(np.float64)
    elif isinstance(data, PIL.Image.Image):
        return np.array(data).astype(np.float64)
    elif isinstance(data, torch.Tensor):
        return data.numpy().astype(np.float64)
    # Add more conversion cases if needed
    else:
        raise ValueError("Unsupported data format")

def check_and_convert_format(updated_raw_dict, new_analyzed_dict):
    converted_raw_dict = {key: convert_to_numpy(value) for key, value in updated_raw_dict.items()}
    converted_analyzed_dict = {key: convert_to_numpy(value) for key, value in new_analyzed_dict.items()}

    return converted_raw_dict, converted_analyzed_dict


In [8]:
import pickle



In [9]:
import torch
torch.cuda.empty_cache()
#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'MAX_SPLIT_SIZE_MB=256'  # Adjust the size as needed

In [10]:
#weight_values = [1.0, 1.5, 2.0, 2.5, 5.0, 10.0, 20.0]  # Example weight values to experiment with

In [11]:
#!pip install tensorboardX
import tensorboardX

In [12]:
import random
# Set a random seed for Python's built-in random module
random_seed = 42
random.seed(random_seed)

# Set a random seed for NumPy
np.random.seed(random_seed)

# Set a random seed for PyTorch on both CPU and CUDA (if available)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

In [13]:
#!pip install tensorboard

In [14]:
import logging
import os
import sys
import tempfile
from glob import glob

In [15]:
import torch
from torch.cuda.amp import autocast, GradScaler
from torch.utils.tensorboard import SummaryWriter
import torch.nn.utils as nn_utils
import torch.autograd as autograd
from torch.optim.lr_scheduler import StepLR

In [16]:
import psutil

In [17]:
import monai

In [18]:
from monai.data import create_test_image_2d, list_data_collate, decollate_batch, DataLoader

In [19]:
from monai.inferers import sliding_window_inference


In [20]:
from monai.metrics import DiceMetric

In [21]:
from monai.transforms import (
    Activations,
    EnsureChannelFirstd,
    Compose,
    ScaleIntensityd,
    ToTensor,
    AsDiscrete
)
#   DivisiblePadd deleted

In [22]:
from monai.transforms import Transform
#from monai.visualize import plot_2d_or_3d_image

In [23]:
from monai.networks.layers import Norm
from monai.transforms import Resized

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [25]:
#!pip install scikit-learn

In [26]:
from sklearn.model_selection import train_test_split
import random


In [27]:
from PIL import Image

In [28]:


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, input_files, segmentation_files, transform=None):
        self.input_files = input_files
        self.segmentation_files = segmentation_files
        self.transform = transform

    def __len__(self):
        return len(self.input_files)

    def __getitem__(self, idx):
        input_path = self.input_files[idx]
        segmentation_path = self.segmentation_files[idx]

        # Load input image (TIFF)
        input_image = Image.open(input_path)
        input_image = np.array(input_image, dtype=np.float32)  # Convert to NumPy array

        # Load segmentation image (TIFF)
        segmentation_image = Image.open(segmentation_path)
        segmentation_image = np.array(segmentation_image, dtype=np.float32)  # Convert to NumPy array

        # Initialize variables
        input_data = input_image
        segmentation_data = segmentation_image

        if self.transform is not None:
            # Apply the specified transforms to input_data and segmentation_data
            transformed_data = self.transform({"img": input_data, "seg": segmentation_data})
            input_data, segmentation_data = transformed_data["img"], transformed_data["seg"]

        return {"img": input_data, "seg": segmentation_data}






max_grad_norm = 1.0
# Define a custom transform to threshold the segmentation tensor
class ThresholdSegmentation(Transform):
    def __init__(self, keys, threshold=0):
        super().__init__()
        self.keys = keys
        self.threshold = threshold

    def __call__(self, data):
        for key in self.keys:
            if key == "seg":
                data[key] = (data[key] > self.threshold).float()
        return data

import matplotlib.pyplot as plt
from monai.transforms import ToNumpy




def main():
    log_dir = "D:/Aytekin/CovidMasterThesis/red_runs/resize512_epoch50_highlr"
    monai.config.print_config()
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)


    #print(len(updated_raw_dict))
    #print(len(new_analyzed_dict))
    # Check and convert data format only once
    #converted_raw_dict, converted_analyzed_dict = check_and_convert_format(updated_raw_dict, new_analyzed_dict)

    # Remove red for now!!!!
    #converted_raw_dict = {k: v for k, v in converted_raw_dict.items() if 'red' not in k}

    # Remove elements where the key contains "red" for converted_analyzed_dict
    #converted_analyzed_dict = {k: v for k, v in converted_analyzed_dict.items() if 'red' not in k}


    # Check if the dictionaries contain the same length or not, then create train val test:
    #if len(converted_raw_dict) != len(converted_analyzed_dict):
    #    raise ValueError("The lengths of converted_raw_dict and converted_analyzed_dict do not match.")

    #num_images = len(converted_raw_dict)
    #print("Length of converted_raw_dict to be used: {}".format(num_images))
    # Calculate the number of images for training, validation, and test, e.g., using an 80-10-10 split

    #
    #converted_raw_dict.keys()
    
    
    

        # define transforms for image and segmentation
    train_transforms = Compose(
        [
            #ToTensor(),
            EnsureChannelFirstd(keys=["img", "seg"], channel_dim="no_channel"),  # Use channel_dim=-1 for NumPy arrays
            ScaleIntensityd(keys=["img", "seg"]),
            Resized(keys=["img", "seg"],spatial_size=(512, 512), mode="nearest"),
            ThresholdSegmentation(keys=["img", "seg"], threshold=0),

            #DivisiblePadd(keys=["img", "seg"],k=16),

        ]
    )
    val_transforms = Compose(
        [
            #ToTensor(),
            EnsureChannelFirstd(keys=["img", "seg"], channel_dim="no_channel"),  # Use channel_dim=-1 for NumPy arrays
            ScaleIntensityd(keys=["img", "seg"]),
            Resized(keys=["img", "seg"],spatial_size=(512, 512), mode="nearest"),
            ThresholdSegmentation(keys=["img", "seg"], threshold=0),
            #DivisiblePadd(keys=["img", "seg"],k=16),


        ]
    )
    
    
    
    


    # Directory paths for input and segmentation images
    input_image_dir = r"E:\Aytekin\TrainingDataRed\Input"
    segmentation_dir = r"E:\Aytekin\TrainingDataRed\Segmentation"
    test_input_dir = r"E:\Aytekin\Data\Test\Input"

    # List of strings to exclude from image names
    exclude_strings = ["12_0500", "29_0520", "01_0520", "08_0420", "29_0820","29_1120","29_1420","29_1720","29_2020","29_2320"]  # Add more strings if needed

    # List all .tif image files in the directories and filter out images with excluded strings
    input_image_files = [os.path.join(input_image_dir, filename) for filename in os.listdir(input_image_dir) if filename.endswith(".tif") and all(exclude not in filename for exclude in exclude_strings)]
    segmentation_files = [os.path.join(segmentation_dir, filename) for filename in os.listdir(segmentation_dir) if filename.endswith(".tif") and all(exclude not in filename for exclude in exclude_strings)]

    # Get the list of files in the "Test Input" folder
    test_input_files = [os.path.splitext(os.path.basename(filename))[0] for filename in os.listdir(test_input_dir) if filename.endswith(".tif")]

    # Exclude images with the same names as those in the "Test Input" folder
    input_image_files = [filename for filename in input_image_files if os.path.splitext(os.path.basename(filename))[0] not in test_input_files]
    segmentation_files = [filename for filename in segmentation_files if os.path.splitext(os.path.basename(filename))[0] not in test_input_files]

    # List all .tif image files in the directories
    input_image_files = [os.path.join(input_image_dir, filename) for filename in os.listdir(input_image_dir) if filename.endswith(".tif")]
    segmentation_files = [os.path.join(segmentation_dir, filename) for filename in os.listdir(segmentation_dir) if filename.endswith(".tif")]

    # Extract file names without extensions
    input_image_names = [os.path.splitext(os.path.basename(filename))[0] for filename in input_image_files]
    segmentation_names = [os.path.splitext(os.path.basename(filename))[0] for filename in segmentation_files]

    # Find the common file names between input and segmentation
    common_names = set(input_image_names) & set(segmentation_names)

    # Filter input_image_files and segmentation_files based on common names
    input_image_files = [filename for filename in input_image_files if os.path.splitext(os.path.basename(filename))[0] in common_names]
    segmentation_files = [filename for filename in segmentation_files if os.path.splitext(os.path.basename(filename))[0] in common_names]




    # Split the data into training and validation sets
    input_image_train, input_image_val, segmentation_train, segmentation_val = train_test_split(
        input_image_files, segmentation_files, test_size=0.2, random_state=42)

    # Create instances of the custom dataset for training and validation
    train_dataset = CustomDataset(input_image_train, segmentation_train, transform=train_transforms)
    val_dataset = CustomDataset(input_image_val, segmentation_val, transform=val_transforms)

    # Define data loaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True, num_workers=0, collate_fn=list_data_collate, pin_memory=torch.cuda.is_available())
    val_loader = DataLoader(val_dataset, batch_size=1, num_workers=0, collate_fn=list_data_collate)








    # define dataset, data loader
    #check_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
    # use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
    #check_loader = DataLoader(check_ds, batch_size=2, num_workers=0, collate_fn=list_data_collate)
    #check_data = monai.utils.misc.first(check_loader)
    #print(check_data["img"].shape, check_data["seg"].shape)
    #print("------\n")
    #print(torch.unique(check_data["seg"]))
    # create a training data loader
    #train_ds = monai.data.Dataset(data=train_files, transform=train_transforms)
    # use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
    #train_loader = DataLoader(
    #    train_ds,
    #    batch_size=2,
    #    shuffle=False,
    #    num_workers=0,
    #    collate_fn=list_data_collate,
    #    pin_memory=torch.cuda.is_available(),
    #)
    # create a validation data loader
    #val_ds = monai.data.Dataset(data=val_files, transform=val_transforms)
    #val_loader = DataLoader(val_ds, batch_size=1, num_workers=0, collate_fn=list_data_collate)
    dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)
    post_trans = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
    # create UNet, DiceLoss and Adam optimizer
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    ######## Different Models Can be Tested ##############
    model = monai.networks.nets.UNet(
        spatial_dims=2,
        in_channels=1,
        out_channels=1,
        channels=(64, 128, 256, 512, 1024),
        strides=(2, 2, 2, 2),
        num_res_units=4,
        norm = Norm.BATCH,
    ).to(device)
    
    
    ######## Different Models Can be Tested ##############
    #model = monai.networks.nets.UNet(
    #    spatial_dims=2,
    #    in_channels=1,
    #    out_channels=1,
    #    channels=(64, 128, 256, 512, 1024, 2048),
    #    strides=(2, 2, 2, 2, 2),
    #    num_res_units=4,
    #    norm=None  # Remove normalization layers
    #).to(device)
    
    loss_function = monai.losses.DiceLoss(include_background=True,sigmoid=True,smooth_nr=1e-05, smooth_dr=1e-05) #, weight = [1/0.9892240500840984, 1/0.01077594991590156]
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2 ) #1e-3
    #optimizer = Novograd(model.parameters(), lr=1e-2, clip=1.0)
    # start a typical PyTorch training
    val_interval = 2
    best_metric = -1
    best_metric_epoch = -1
    epoch_loss_values = list()
    metric_values = list()
    writer = SummaryWriter(log_dir=log_dir)


    autograd.set_detect_anomaly(True)
    scaler = GradScaler()

    # Define the step size and gamma (decay factor)
    step_size = 1000  # Adjust this according to your training schedule
    gamma = 0.5     # Adjust this according to your preference

    # Create a learning rate scheduler
    scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)
    # Create a DiceMetric instance for training
    train_dice_metric = DiceMetric(include_background=False, reduction="mean", get_not_nans=False)

    for epoch in range(50):
        print("-" * 50)
        print(f"epoch {epoch + 1}/{50}")
        model.train()
        epoch_loss = 0
        step = 0
        # Clear the DiceMetric for the new epoch
        train_dice_metric.reset()
        for batch_data in train_loader:
            step += 1
            inputs, labels = batch_data["img"].to(device), batch_data["seg"].to(device)
            optimizer.zero_grad()
            with autocast():
                outputs = model(inputs)
                loss = loss_function(outputs, labels)

            # Calculate the Dice score during training
            post_outputs = post_trans(outputs)
            train_dice_metric(y_pred=post_outputs, y=labels)

            scaler.scale(loss).backward()
            #scaler.unscale_(optimizer)  # Unscales the gradients of optimizer's assigned params in-place
            ############## Gradient Clipping Part ###################
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)  # Add this line to clip gradients
            scaler.step(optimizer)
            scaler.update()

            epoch_loss += loss.item()
            epoch_len = len(train_dataset) // train_loader.batch_size
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)




        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        # Calculate the average Dice score for the epoch
        train_dice_score = train_dice_metric.aggregate().item()
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}, train_dice: {train_dice_score:.4f}")
        writer.add_scalar("train_dice", train_dice_score, epoch + 1)


        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")
        scheduler.step()
        if (epoch + 1) % val_interval == 0:
            model.eval()
            with torch.no_grad():
                val_images = None
                val_labels = None
                val_outputs = None
                for val_data in val_loader:
                    val_images, val_labels = val_data["img"].to(device), val_data["seg"].to(device)
                    roi_size = (96, 96)
                    sw_batch_size = 4
                    val_outputs = sliding_window_inference(val_images, roi_size, sw_batch_size, model)
                    val_outputs = [post_trans(i) for i in decollate_batch(val_outputs)]
                    # compute metric for current iteration
                    dice_metric(y_pred=val_outputs, y=val_labels)

                    for i in range(len(val_images)):
                        input_image = val_images[i:i + 1]  # Get the i-th image
                        ground_truth = val_labels[i:i + 1]  # Get the i-th ground truth
                        prediction = val_outputs[i:i + 1]  # Get the i-th prediction

                        # Perform intensity scaling if needed
                        #a_min = 0.0
                        #a_max = 255.0
                        #b_min = 0.0
                        #b_max = 1.0

                        if isinstance(prediction, list):
                            prediction = prediction[0]
                        
                        
                        
                        
                        writer.add_image('Validation/Input', input_image, global_step=epoch,dataformats='NCHW')
                        writer.add_image('Validation/GroundTruth', ground_truth, global_step=epoch,dataformats='NCHW')
                        writer.add_image('Validation/Prediction', prediction, global_step=epoch,dataformats='CHW')


                # aggregate the final mean dice result
                metric = dice_metric.aggregate().item()
                # reset the status for next validation round
                dice_metric.reset()
                metric_values.append(metric)
                if metric > best_metric:
                    best_metric = metric
                    best_metric_epoch = epoch + 1
                    torch.save(model.state_dict(), "512red_alldata_seg2d_epoch50_highlr.pth")
                    print("saved new best metric model")
                print(
                    "current epoch: {} current mean dice: {:.4f} best mean dice: {:.4f} at epoch {}".format(
                        epoch + 1, metric, best_metric, best_metric_epoch
                    )
                )
                writer.add_scalar("val_mean_dice", metric, epoch + 1)
                # plot the last model output as GIF image in TensorBoard with the corresponding image and label




    autograd.set_detect_anomaly(False)
    writer.close()



    


if __name__ == "__main__":
    main()

MONAI version: 1.2.0
Numpy version: 1.26.0
Pytorch version: 2.0.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: c33f1ba588ee00229a309000e888f9817b4f1934
MONAI __file__: C:\Users\ge59gor\AppData\Local\anaconda3\envs\deepsight\Lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: NOT INSTALLED or UNKNOWN VERSION.
scikit-image version: 0.22.0
Pillow version: 9.4.0
Tensorboard version: 2.14.1
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.2
tqdm version: NOT INSTALLED or UNKNOWN VERSION.
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 2.0.3
einops version: NOT INSTALLED or UNKNOWN VERSION.
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.

For details ab

246/1088, train_loss: 0.0459
247/1088, train_loss: 0.0426
248/1088, train_loss: 0.0467
249/1088, train_loss: 0.0439
250/1088, train_loss: 0.0427
251/1088, train_loss: 0.0670
252/1088, train_loss: 0.0882
253/1088, train_loss: 0.0614
254/1088, train_loss: 0.0396
255/1088, train_loss: 0.0515
256/1088, train_loss: 0.0506
257/1088, train_loss: 0.0537
258/1088, train_loss: 0.0477
259/1088, train_loss: 0.0466
260/1088, train_loss: 0.0507
261/1088, train_loss: 0.0501
262/1088, train_loss: 0.0559
263/1088, train_loss: 0.0433
264/1088, train_loss: 0.0451
265/1088, train_loss: 0.0421
266/1088, train_loss: 0.0592
267/1088, train_loss: 0.0564
268/1088, train_loss: 0.0460
269/1088, train_loss: 0.0485
270/1088, train_loss: 0.0429
271/1088, train_loss: 0.0435
272/1088, train_loss: 0.0452
273/1088, train_loss: 0.0559
274/1088, train_loss: 0.0513
275/1088, train_loss: 0.0453
276/1088, train_loss: 0.0448
277/1088, train_loss: 0.0427
278/1088, train_loss: 0.0454
279/1088, train_loss: 0.0500
280/1088, trai

529/1088, train_loss: 0.0378
530/1088, train_loss: 0.0417
531/1088, train_loss: 0.0455
532/1088, train_loss: 0.0360
533/1088, train_loss: 0.0543
534/1088, train_loss: 0.0369
535/1088, train_loss: 0.0525
536/1088, train_loss: 0.0354
537/1088, train_loss: 0.0393
538/1088, train_loss: 0.0372
539/1088, train_loss: 0.0396
540/1088, train_loss: 0.0394
541/1088, train_loss: 0.0381
542/1088, train_loss: 0.0387
543/1088, train_loss: 0.0399
544/1088, train_loss: 0.0416
545/1088, train_loss: 0.0391
546/1088, train_loss: 0.0457
547/1088, train_loss: 0.0475
548/1088, train_loss: 0.0368
549/1088, train_loss: 0.0533
550/1088, train_loss: 0.0465
551/1088, train_loss: 0.0393
552/1088, train_loss: 0.0390
553/1088, train_loss: 0.0375
554/1088, train_loss: 0.0376
555/1088, train_loss: 0.0379
556/1088, train_loss: 0.0382
557/1088, train_loss: 0.0372
558/1088, train_loss: 0.0459
559/1088, train_loss: 0.0431
560/1088, train_loss: 0.0515
561/1088, train_loss: 0.0453
562/1088, train_loss: 0.0400
563/1088, trai

812/1088, train_loss: 0.0344
813/1088, train_loss: 0.0433
814/1088, train_loss: 0.0363
815/1088, train_loss: 0.0438
816/1088, train_loss: 0.0349
817/1088, train_loss: 0.0393
818/1088, train_loss: 0.0369
819/1088, train_loss: 0.0458
820/1088, train_loss: 0.0361
821/1088, train_loss: 0.0424
822/1088, train_loss: 0.0342
823/1088, train_loss: 0.0345
824/1088, train_loss: 0.0341
825/1088, train_loss: 0.0359
826/1088, train_loss: 0.0359
827/1088, train_loss: 0.0345
828/1088, train_loss: 0.0336
829/1088, train_loss: 0.0393
830/1088, train_loss: 0.0435
831/1088, train_loss: 0.0331
832/1088, train_loss: 0.0373
833/1088, train_loss: 0.0394
834/1088, train_loss: 0.0360
835/1088, train_loss: 0.0429
836/1088, train_loss: 0.0359
837/1088, train_loss: 0.0350
838/1088, train_loss: 0.0335
839/1088, train_loss: 0.0453
840/1088, train_loss: 0.0440
841/1088, train_loss: 0.0352
842/1088, train_loss: 0.0366
843/1088, train_loss: 0.0390
844/1088, train_loss: 0.0368
845/1088, train_loss: 0.0348
846/1088, trai

1/1088, train_loss: 0.0397
2/1088, train_loss: 0.0373
3/1088, train_loss: 0.0352
4/1088, train_loss: 0.0369
5/1088, train_loss: 0.0336
6/1088, train_loss: 0.0329
7/1088, train_loss: 0.0346
8/1088, train_loss: 0.0327
9/1088, train_loss: 0.0317
10/1088, train_loss: 0.0315
11/1088, train_loss: 0.0403
12/1088, train_loss: 0.0396
13/1088, train_loss: 0.0339
14/1088, train_loss: 0.0343
15/1088, train_loss: 0.0462
16/1088, train_loss: 0.0380
17/1088, train_loss: 0.0353
18/1088, train_loss: 0.0359
19/1088, train_loss: 0.0389
20/1088, train_loss: 0.0375
21/1088, train_loss: 0.0373
22/1088, train_loss: 0.0350
23/1088, train_loss: 0.0355
24/1088, train_loss: 0.0415
25/1088, train_loss: 0.0344
26/1088, train_loss: 0.0346
27/1088, train_loss: 0.0335
28/1088, train_loss: 0.0355
29/1088, train_loss: 0.0315
30/1088, train_loss: 0.0331
31/1088, train_loss: 0.0364
32/1088, train_loss: 0.0469
33/1088, train_loss: 0.0528
34/1088, train_loss: 0.0464
35/1088, train_loss: 0.0356
36/1088, train_loss: 0.0363
3

...

1087/1088, train_loss: 0.0314
1088/1088, train_loss: 0.0290
1089/1088, train_loss: 0.0268
epoch 16 average loss: 0.0287, train_dice: 0.9713
epoch 16 average loss: 0.0287
current epoch: 16 current mean dice: 0.9690 best mean dice: 0.9703 at epoch 8
--------------------------------------------------
epoch 17/50
1/1088, train_loss: 0.0265
2/1088, train_loss: 0.0271
3/1088, train_loss: 0.0252
4/1088, train_loss: 0.0284
5/1088, train_loss: 0.0293
6/1088, train_loss: 0.0262
7/1088, train_loss: 0.0281
8/1088, train_loss: 0.0297
9/1088, train_loss: 0.0294
10/1088, train_loss: 0.0279
11/1088, train_loss: 0.0298
12/1088, train_loss: 0.0266
13/1088, train_loss: 0.0291
14/1088, train_loss: 0.0296
15/1088, train_loss: 0.0291
16/1088, train_loss: 0.0262
17/1088, train_loss: 0.0281
18/1088, train_loss: 0.0282
19/1088, train_loss: 0.0296
20/1088, train_loss: 0.0285
21/1088, train_loss: 0.0275
22/1088, train_loss: 0.0283
23/1088, train_loss: 0.0290
24/1088, train_loss: 0.0267
25/1088, train_loss: 0.027

277/1088, train_loss: 0.0384
278/1088, train_loss: 0.0268
279/1088, train_loss: 0.0281
280/1088, train_loss: 0.0273
281/1088, train_loss: 0.0280
282/1088, train_loss: 0.0286
283/1088, train_loss: 0.0280
284/1088, train_loss: 0.0294
285/1088, train_loss: 0.0283
286/1088, train_loss: 0.0286
287/1088, train_loss: 0.0286
288/1088, train_loss: 0.0272
289/1088, train_loss: 0.0303
290/1088, train_loss: 0.0368
291/1088, train_loss: 0.0278
292/1088, train_loss: 0.0276
293/1088, train_loss: 0.0271
294/1088, train_loss: 0.0290
295/1088, train_loss: 0.0249
296/1088, train_loss: 0.0257
297/1088, train_loss: 0.0259
298/1088, train_loss: 0.0288
299/1088, train_loss: 0.0272
300/1088, train_loss: 0.0281
301/1088, train_loss: 0.0304
302/1088, train_loss: 0.0289
303/1088, train_loss: 0.0257
304/1088, train_loss: 0.0293
305/1088, train_loss: 0.0258
306/1088, train_loss: 0.0276
307/1088, train_loss: 0.0269
308/1088, train_loss: 0.0280
309/1088, train_loss: 0.0301
310/1088, train_loss: 0.0275
311/1088, trai

560/1088, train_loss: 0.0298
561/1088, train_loss: 0.0270
562/1088, train_loss: 0.0279
563/1088, train_loss: 0.0282
564/1088, train_loss: 0.0272
565/1088, train_loss: 0.0282
566/1088, train_loss: 0.0274
567/1088, train_loss: 0.0294
568/1088, train_loss: 0.0287
569/1088, train_loss: 0.0279
570/1088, train_loss: 0.0266
571/1088, train_loss: 0.0279
572/1088, train_loss: 0.0300
573/1088, train_loss: 0.0271
574/1088, train_loss: 0.0288
575/1088, train_loss: 0.0290
576/1088, train_loss: 0.0281
577/1088, train_loss: 0.0265
578/1088, train_loss: 0.0282
579/1088, train_loss: 0.0298
580/1088, train_loss: 0.0282
581/1088, train_loss: 0.0272
582/1088, train_loss: 0.0290
583/1088, train_loss: 0.0248
584/1088, train_loss: 0.0264
585/1088, train_loss: 0.0287
586/1088, train_loss: 0.0280
587/1088, train_loss: 0.0275
588/1088, train_loss: 0.0279
589/1088, train_loss: 0.0368
590/1088, train_loss: 0.0269
591/1088, train_loss: 0.0279
592/1088, train_loss: 0.0261
593/1088, train_loss: 0.0275
594/1088, trai

843/1088, train_loss: 0.0288
844/1088, train_loss: 0.0263
845/1088, train_loss: 0.0306
846/1088, train_loss: 0.0277
847/1088, train_loss: 0.0301
848/1088, train_loss: 0.0280
849/1088, train_loss: 0.0267
850/1088, train_loss: 0.0295
851/1088, train_loss: 0.0284
852/1088, train_loss: 0.0335
853/1088, train_loss: 0.0264
854/1088, train_loss: 0.0272
855/1088, train_loss: 0.0252
856/1088, train_loss: 0.0284
857/1088, train_loss: 0.0306
858/1088, train_loss: 0.0296
859/1088, train_loss: 0.0282
860/1088, train_loss: 0.0309
861/1088, train_loss: 0.0284
862/1088, train_loss: 0.0290
863/1088, train_loss: 0.0286
864/1088, train_loss: 0.0283
865/1088, train_loss: 0.0304
866/1088, train_loss: 0.0279
867/1088, train_loss: 0.0259
868/1088, train_loss: 0.0303
869/1088, train_loss: 0.0265
870/1088, train_loss: 0.0292
871/1088, train_loss: 0.0292
872/1088, train_loss: 0.0292
873/1088, train_loss: 0.0298
874/1088, train_loss: 0.0275
875/1088, train_loss: 0.0253
876/1088, train_loss: 0.0293
877/1088, trai

KeyboardInterrupt: 

In [30]:
!tensorboard --logdir=D:/Aytekin/CovidMasterThesis/red_runs/resize512_epoch50_highlr --port=6017

^C


In [ ]:
print(a)

In [ ]:
# import os
#os.system("pkill -f 'tensorboard'")

In [ ]:
#%load_ext tensorboard

In [ ]:
#!pip install --upgrade tensorboard